# Install packages


In [1]:
%%capture --no-stderr
!pip install -q langchain_google_genai langchain_core langchain_community tavily-python pydantic
%pip install --quiet -U langgraph

# Import packages

In [6]:
from pydantic import BaseModel
from langgraph.graph import MessagesState
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage

 # Set up authentication keys

In [3]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

In [4]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash",
    api_key = GEMINI_API_KEY,
    temperature = 0)

In [5]:
llm.invoke("hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--804272ce-1a51-45fb-81bd-74cf74264841-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [ ]:
import pandas as pd
import psycopg2

class MessagesState(MessagesState):
    # Add any keys needed beyond messages, which is pre-built
    pass

def get_data_from_db(client, query) -> :
  """
  This function is used to get data from the db and return it
  the database name is cryptofocus the schema name is public and the table name is configs_strategies
  the configs strategies contian the colums are name symbol tim_horizon data_exchange

  Args
  client :
  query ;

  return
  data
  """
  data = pd.read_sql(f'{query}', client)
  return data

def create_db_connection() - > :
  """
  this futnion can return the db client
  args: None
  return : client
  """
	host = os.getenv('DB_HOST')
	user_name = os.getenv('DB_USER')
	password = os.getenv('DB_PASSWORD')
	db_name = os.getenv('DB_NAME')

	connection = psycopg2.connect(
		host=host,
		database=db_name,
		user=user_name,
		password=password)
	return connection

# Bind tools to the model
tools = [get_data_from_db, create_db_connection]
llm_with_tools = llm.bind_tools(tools)


def tool_calling_llm(state: MessagesState) -> MessagesState:
    """Node that invokes the LLM with tool capabilities"""
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

# Build tool router grapgh
builder = StateGraph(MessagesState)
builder.add_node(tool_calling_llm)
builder.add_node("tools", ToolNode(tools))


# Add routing logic
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition)
builder.add_edge("tools", END)

tool_router = builder.compile()
display(Image(tool_router.get_graph().draw_mermaid_png()))
result = tool_router.invoke({"message": "give me all the data where the timhorizon is 4h"})